# Save a small version of the video with the trajectory on it

#### Written by Matan 20230306

In [1]:
import cv2 as cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import trackpy as tp
from ipywidgets import HBox, Textarea, interact
import time
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def l2n(x):
    return np.array(x)

def cropImage(img,x0,y0,width,height):
    return img[y0:y0+height,x0:x0+width,:]

def rebin(arr,new_shape):
    shape = (new_shape[0], arr.shape[0] // new_shape[0],
             new_shape[1], arr.shape[1] // new_shape[1],
             new_shape[2], arr.shape[2] // new_shape[2])
    
    return arr.reshape(shape).mean(-1).mean(1).mean(2)

def timeToSec(timeStr):#Transform m:s format into total number of seconds
    timeArr = timeStr.split(':')
    timeMin = int(timeArr[0])
    timeSec = int(timeArr[1])
    totSec = timeMin*60+timeSec
    return totSec

In [2]:
expSumFileName = 'transportExpSumMaster.csv'
expSum = pd.read_csv(expSumFileName,index_col=0)

In [4]:
videosToMinify = # Choose only some of the experimental list


### Output video parameters

In [4]:
# Ccale down factor
scaleDownFactor = 4

# Speed up (frame skip)
speedUpFactor = 3 

# Suffix to add to output video
smallVideoSuffix = '_small.mp4'

In [5]:

width,height = 1200,1080
x0,y0 = 300,0
widthScaled = width//scaleDownFactor
heightScaled = height//scaleDownFactor

startTimeGlobal = time.time()

print('Minifying ' + str(len(videosToMinify)).zfill(3) + ' videos.')
for index,row in videosToMinify.iterrows():   
    startTime = time.time()
    
    
    
    ############## Fetch experiments data #################
    fileNameVideo = row.path
    fileNameLinked = row.linkingFileName
    
    startMin = row.start_min
    endMin = row.end_min
    
    ############## load experiments files #################
    # Load video file
    video = cv2.VideoCapture(fileNameVideo)
    # Load locating file

    fps = video.get(cv2.CAP_PROP_FPS) 
    
    ########### Filename to save small video #############
    outputFileName = fileNameVideo+smallVideoSuffix
    #Compute start and end frame
    startSec = timeToSec(startMin)
    endSec = timeToSec(endMin)
    startFrame = int(startSec*fps)
    endFrame = int(endSec*fps)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(outputFileName,fourcc, fps, (widthScaled,heightScaled))
    video.set(cv2.CAP_PROP_POS_FRAMES, startFrame)
    frameNum = video.get(cv2.CAP_PROP_POS_FRAMES)
    ret = True
    print('\tScaling expe :\t#' + str(index).zfill(3))
    print('\tOriginal path:\t' +  fileNameVideo)
    print('\tFrame count  :\t' + str(endFrame-startFrame) +' frames')
    print('\tMinified dest:\t' + outputFileName)
    while ret and frameNum<endFrame:
        #video.grab()#captureFrame(video,frameNum)
        for i in range(speedUpFactor): #skip frames by speedUpFactor
            ret, img = video.read()

        if(ret): #if succesfffuly read an image
            frameNum = video.get(cv2.CAP_PROP_POS_FRAMES)
            #imgRed = img[:,:,0]

            #For large images, locate a cropped window
            imgCropped = cropImage(img,x0,y0,width,height)
            imgScaled = cv2.resize(imgCropped,(widthScaled,heightScaled),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
            #Edge detection:
            out.write(imgScaled)
            #df = pd.concat([df,dft],ignore_index=True)

    video.release()
    out.release()

    endTime = time.time()
    duration = endTime-startTime
    print('\tScaling and cropping '+ fileNameVideo + ' with '+ str(frameNum-startFrame) + ' frames took ' + str(duration) + ' seconds.\n')
endTimeGlobal = time.time()
durationGlobal = endTimeGlobal-startTimeGlobal

print('\nScaling and cropping ' + str(len(videosToMinify)) + ' videos took ' + str(np.round(durationGlobal/60,2)) + ' minutes')

Minifying 044 videos.
	Scaling expe :	#042
	Original path:	D:\Eden\5_1_23\fronters1passive28cm\C0556.MP4
	Frame count  :	30125 frames
	Minified dest:	D:\Eden\5_1_23\fronters1passive28cm\C0556.MP4_small.mp4
	Scaling and cropping D:\Eden\5_1_23\fronters1passive28cm\C0556.MP4 with 30125.0 frames took 138.56393790245056 seconds.

	Scaling expe :	#043
	Original path:	D:\Eden\5_1_23\fronters1passive28cm\C0555.MP4
	Frame count  :	31775 frames
	Minified dest:	D:\Eden\5_1_23\fronters1passive28cm\C0555.MP4_small.mp4
	Scaling and cropping D:\Eden\5_1_23\fronters1passive28cm\C0555.MP4 with 31775.0 frames took 165.91434359550476 seconds.

	Scaling expe :	#044
	Original path:	D:\Eden\5_1_23\fronters1passive28cm\C0554.MP4
	Frame count  :	30050 frames
	Minified dest:	D:\Eden\5_1_23\fronters1passive28cm\C0554.MP4_small.mp4
	Scaling and cropping D:\Eden\5_1_23\fronters1passive28cm\C0554.MP4 with 30050.0 frames took 183.73223781585693 seconds.

	Scaling expe :	#045
	Original path:	D:\Eden\5_1_23\fronters